### Item 8: Handle nulls properly
`null` means a lack of value.

property 라면, 값이 초기화되지 않았거나, 삭제됨을 의미한다.

function 이라면, 함수마다 다르게 해석된다.
- `String.toIntOrNull()` 은 `String` 이 `Int` 로 파싱되지 못할 때 `null` 을 반환한다.
- `Iterable<T>.firstOrNull(() -> Boolean)` 은 어떠한 element 도 predicate 를 만족하지 못할 때 `null` 을 반환한다.

`null` 의 의미는 언제나 가능한 명확해야 한다.

`null` 은 보통 다음 세 가지 방법으로 처리한다.
- safe call `?.`, Elvis operator `?:` 등을 활용한 `null` 핸들링
- Throw error
- nullable 이 아닌 타입을 반환하는 함수로 리팩터링

#### Throw an error
`null` 이 되지 않을 것이란 확신이 있다면, error 를 throw 함으로써 왜 `null` 이 발생했는지 정보를 얻을 필요가 있다.

이 때, 세 가지 방법이 있다.
- throw error
- not-null assertion `!!`
- `requireNotNull`, `checkNotNull`

#### The problems with the not-null assertion !!
`!!` 연산자 사용은 피하는 것이 좋다.

`!!` 이 주는 정보는 거의 없기 때문에 실효성이 떨어짐.

`!!` 은 `null` 이 발생하지 않을 것으로 예상되는 nullable 타입에 사용한다.

문제는 현재에 비록 `null` 이 예상되지 않는다고 해도, 미래엔 `null` 이 발생할 위험을 숨기고 있다는 것이다.

보통 not-null assertion 은 적절하지 않지만, 다음의 경우 예외적으로 사용될 수 있다.
- nullability 가 제대로 표현되지 않은 라이브러리를 사용할 때

위 경우를 거꾸로 말하면 kotlin API 를 설계할 때, nullability 를 제대로 표현해서 `!!` 필요하지 않도록 해야 한다.

#### Avoiding meaningless nullability
nullability 는 핸들링 비용이 들기 때문에, 필요한 경우가 아니라면 사용하지 않는 것이 좋다.

Empty Collection 대신 `null` 를 return 하지 마라.
- `null` 은 Collection 자체가 없다는 것을 의미한다.
- Empty Collection 은 Collection 에 elements 가 없다는 것을 의미한다.

`nullable enum` vs `None enum`
- `nullable enum`: `null` 은 핸들링 되어야하는 특별한 메시지
- `None enum`: 정의에 없음을 의미하며, 필요한 경우 사용하는 쪽에서 추가해서 활용할 수 있음.

#### `lateinit` property and `notNull` delegate
`lateinit` 은  property 가 사용되기 전에 초기화 될 것이란 확신이 있을 때 사용하기 좋은 방법이다.
- unpack 할 필요가 없다.
- 예상치 못하게 만약 초기화되지 않았다면, 정보를 Exception 에 담아 알려준다.
```kotlin
class UserControllerTest {
    private lateinit var dao: UserDao // 추후 초기화식 필요
}
```

`lateinit` 을 사용하지 못하는 경우
- Int, Long, Double, Boolean 과 같이 JVM 의 primitive type 과 연결된 타입으로 property 를 초기화해야 하는 경우

이런 경우, `Delegates.notNull` 을 대신 사용할 수 있으나, 속도는 상대적으로 느리다.
```kotlin
class DoctorActivity: Activity() {
    private var doctorId: Int by Delegates.notNull() // 추후 초기화식 필요
    
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        doctorId = intent.extras.getInt(DOCTOR_ID_ARG)
    }
}

class DoctorActivity: Activity() {
    private var doctorId: Int by arg(DOCTOR_ID_ARG)
}
```